# 05b. Prompt Tuning
## Synthetic Instruction Tuner - Alternative Adaptation Method

This notebook implements **Prompt Tuning** as an alternative adaptation method for comparison:
1. Load base model (same as SFT)
2. Initialize soft prompt embeddings
3. Train only the prompt parameters
4. Collect efficiency metrics for comparison
5. Evaluate and save the model

**Comparison with LoRA**:
| Method | Trainable Params | Memory | Training Time |
|--------|-----------------|--------|---------------|
| LoRA (r=8) | ~0.67% | Medium | 2-4 hours (A100) |
| Prompt Tuning | ~0.01% | Low | 1-2 hours (A100) |

**Training settings (A100 GPU optimized)**:
- Virtual tokens: 20
- Batch size: 12 (A100 40GB VRAM)
- Gradient accumulation: 2 (Effective batch size: 24)
- **BF16 enabled** for optimal A100 performance

**Expected runtime on A100**: 1-2 hours
**Cost**: ~5-10 compute units

**⚠️ IMPORTANT: Before running this notebook**:
1. Runtime → Change runtime type → GPU type: **A100**
2. Runtime → Restart runtime
3. Run `05_sft_training.ipynb` first for comparison metrics

## 1. Setup & Efficiency Tracking

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Project path
PROJECT_ROOT = "/content/drive/MyDrive/synthetic-instruction-tuner"

Mounted at /content/drive


In [26]:
# Load configuration
import json

with open(f"{PROJECT_ROOT}/config.json", 'r') as f:
    config = json.load(f)

print("Configuration loaded!")

Configuration loaded!


In [27]:
# Configure for A100 GPU (use same settings as SFT for fair comparison)
with open(f"{PROJECT_ROOT}/config.json", 'r') as f:
    config = json.load(f)

# A100 optimization settings (same as SFT)
config['training']['prompt_tuning_batch_size'] = 12
config['training']['prompt_tuning_gradient_accumulation'] = 2

# Save updated config
with open(f"{PROJECT_ROOT}/config.json", 'w') as f:
    json.dump(config, f, indent=2)

print("✅ A100 GPU settings applied!")
print(f"  Batch size: {config['training']['prompt_tuning_batch_size']}")
print(f"  Gradient accumulation: {config['training']['prompt_tuning_gradient_accumulation']}")
print(f"  Effective batch size: {config['training']['prompt_tuning_batch_size'] * config['training']['prompt_tuning_gradient_accumulation']}")

✅ A100 GPU settings applied!
  Batch size: 12
  Gradient accumulation: 2
  Effective batch size: 24


In [28]:
# Install libraries
!pip install -q --upgrade transformers>=4.41.0 peft>=0.7.0 trl>=0.7.4 datasets>=2.16.0 accelerate>=0.25.0 bitsandbytes>=0.41.3

print("✅ Libraries installed successfully!")

✅ Libraries installed successfully!


In [30]:
import torch
import json
import os
import time
from datetime import datetime
from datasets import Dataset
import gc

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name}")
    print(f"GPU Memory: {gpu_memory:.2f} GB")

    # Verify A100
    if "A100" not in gpu_name:
        print("\n⚠️ WARNING: This notebook is optimized for A100 GPU!")
        print(f"   Current GPU: {gpu_name}")
        print("   Please change runtime to A100:")
        print("   Runtime → Change runtime type → GPU type: A100")
    else:
        print("\n✅ A100 GPU detected! Ready for training.")

PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: NVIDIA A100-SXM4-80GB
GPU Memory: 85.17 GB

✅ A100 GPU detected! Ready for training.


In [31]:
# A100 GPU optimization settings
import torch

# Enable BF16 for A100 (supported natively)
torch.backends.cuda.matmul.allow_bf16_reduced_precision_reduction = True
torch.backends.cudnn.allow_tf32 = True

print("=" * 50)
print("A100 GPU Performance Settings:")
print(f"  BF16 enabled: {torch.backends.cuda.matmul.allow_bf16_reduced_precision_reduction}")
print(f"  TF32 enabled: {torch.backends.cudnn.allow_tf32}")
print(f"  Optimal for A100 40GB VRAM")
print("=" * 50)

A100 GPU Performance Settings:
  BF16 enabled: True
  TF32 enabled: True
  Optimal for A100 40GB VRAM


In [32]:
# Efficiency Metrics Tracker
class EfficiencyTracker:
    """Track efficiency metrics for adaptation method comparison."""

    def __init__(self, method_name: str):
        self.method_name = method_name
        self.metrics = {
            "method": method_name,
            "memory_allocated_gb": [],
            "memory_reserved_gb": [],
            "training_time_seconds": 0,
            "trainable_params": 0,
            "total_params": 0,
            "trainable_ratio": 0,
            "inference_tokens_per_sec": 0,
        }
        self.start_time = None

    def log_memory(self):
        """Log current GPU memory usage."""
        if torch.cuda.is_available():
            allocated = torch.cuda.memory_allocated() / 1e9
            reserved = torch.cuda.memory_reserved() / 1e9
            self.metrics["memory_allocated_gb"].append(allocated)
            self.metrics["memory_reserved_gb"].append(reserved)
            return {"allocated": allocated, "reserved": reserved}
        return None

    def start_training(self):
        """Start timing training."""
        self.start_time = time.time()
        self.log_memory()

    def end_training(self):
        """End timing training."""
        if self.start_time:
            self.metrics["training_time_seconds"] = time.time() - self.start_time
        self.log_memory()

    def log_params(self, model):
        """Log parameter counts."""
        trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
        total = sum(p.numel() for p in model.parameters())
        self.metrics["trainable_params"] = trainable
        self.metrics["total_params"] = total
        self.metrics["trainable_ratio"] = trainable / total if total > 0 else 0

    def log_inference_speed(self, tokens_generated: int, time_taken: float):
        """Log inference speed."""
        self.metrics["inference_tokens_per_sec"] = tokens_generated / time_taken if time_taken > 0 else 0

    def get_summary(self):
        """Get summary metrics."""
        summary = {
            "method": self.method_name,
            "trainable_params": self.metrics["trainable_params"],
            "total_params": self.metrics["total_params"],
            "trainable_ratio_percent": self.metrics["trainable_ratio"] * 100,
            "peak_memory_gb": max(self.metrics["memory_allocated_gb"]) if self.metrics["memory_allocated_gb"] else 0,
            "training_time_hours": self.metrics["training_time_seconds"] / 3600,
            "inference_tokens_per_sec": self.metrics["inference_tokens_per_sec"],
        }
        return summary

    def save(self, path: str):
        """Save metrics to JSON."""
        with open(path, 'w') as f:
            json.dump(self.get_summary(), f, indent=2)
        print(f"Metrics saved to {path}")

# Initialize tracker
tracker = EfficiencyTracker("prompt_tuning")
print("Efficiency tracker initialized!")

Efficiency tracker initialized!


## 2. Load Training Data

In [33]:
# Load SFT training data (same as LoRA for fair comparison)
TRAIN_PATH = f"{config['paths']['data_filtered']}/sft_train.json"
VAL_PATH = f"{config['paths']['data_filtered']}/sft_val.json"

with open(TRAIN_PATH, 'r', encoding='utf-8') as f:
    train_data = json.load(f)

with open(VAL_PATH, 'r', encoding='utf-8') as f:
    val_data = json.load(f)

print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")

Training samples: 900
Validation samples: 100


In [54]:
# Convert to HuggingFace Dataset
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

print(f"Train dataset: {train_dataset}")
print(f"Val dataset: {val_dataset}")

Train dataset: Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 900
})
Val dataset: Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 100
})


## 3. Load Base Model

In [55]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, Trainer, DataCollatorForLanguageModeling


# Use same base model as SFT for fair comparison
BASE_MODEL_ID = config['models']['sft_base']
OUTPUT_DIR = f"{config['paths']['models_prompt_tuning']}/checkpoint"

print(f"Loading base model: {BASE_MODEL_ID}")
print(f"Output directory: {OUTPUT_DIR}")

Loading base model: meta-llama/Llama-3.2-3B
Output directory: /content/drive/MyDrive/synthetic-instruction-tuner/models/prompt_tuning/checkpoint


In [57]:
# 4-bit quantization (A100 with BF16 - same as SFT for fair comparison)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,  # BF16 for A100
    bnb_4bit_use_double_quant=True,
)

print("✅ Quantization config created (BF16 for A100)")

✅ Quantization config created (BF16 for A100)


In [58]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)

# Set pad token if not present
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer.padding_side = "right"

print(f"Tokenizer loaded. Vocab size: {tokenizer.vocab_size}")

Tokenizer loaded. Vocab size: 128000


In [59]:
# Load model
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model.config.use_cache = False

# Log initial memory
mem = tracker.log_memory()
print(f"Model loaded!")
print(f"GPU Memory: {mem['allocated']:.2f} GB allocated, {mem['reserved']:.2f} GB reserved")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded!
GPU Memory: 4.49 GB allocated, 6.98 GB reserved


## 4. Configure Prompt Tuning

In [61]:
from peft import PromptTuningConfig, PromptTuningInit, get_peft_model, TaskType

# Prompt Tuning configuration
# num_virtual_tokens: number of soft prompt tokens to prepend
prompt_tuning_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    num_virtual_tokens=config['training']['prompt_tuning_num_virtual_tokens'],
    prompt_tuning_init=PromptTuningInit.RANDOM,
    tokenizer_name_or_path=BASE_MODEL_ID,
)

print("Prompt Tuning config:")
print(f"  num_virtual_tokens: {prompt_tuning_config.num_virtual_tokens}")
print(f"  init_method: {prompt_tuning_config.prompt_tuning_init}")

Prompt Tuning config:
  num_virtual_tokens: 20
  init_method: PromptTuningInit.RANDOM


In [62]:
# Apply Prompt Tuning to model
model = get_peft_model(model, prompt_tuning_config)

# Log parameters
tracker.log_params(model)

print(f"\nTrainable parameters: {tracker.metrics['trainable_params']:,}")
print(f"Total parameters: {tracker.metrics['total_params']:,}")
print(f"Trainable ratio: {tracker.metrics['trainable_ratio']*100:.4f}%")

# Compare with LoRA
print("\n--- Comparison ---")
print("Prompt Tuning trains ONLY the soft prompt embeddings")
print("This is typically 10-100x fewer parameters than LoRA!")


Trainable parameters: 61,440
Total parameters: 1,803,525,120
Trainable ratio: 0.0034%

--- Comparison ---
Prompt Tuning trains ONLY the soft prompt embeddings
This is typically 10-100x fewer parameters than LoRA!


In [63]:
# Visualize trainable parameters
print("\nTrainable layers:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"  {name}: {param.shape}")


Trainable layers:
  prompt_encoder.default.embedding.weight: torch.Size([20, 3072])


## 5. Format Training Data

In [64]:
def format_and_tokenize(sample):
    """Format and tokenize instruction-response pair for training."""
    instruction = sample["instruction"]
    response = sample["output"]

    # Same format as LoRA for fair comparison
    text = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{response}<|eot_id|>"

    # Tokenize with padding to max_length
    tokenized = tokenizer(
        text,
        truncation=True,
        max_length=config['training']['sft_max_seq_length'],
        padding="max_length",  # 패딩 활성화
        return_tensors=None,
    )

    # Add labels (same as input_ids for causal LM)
    tokenized["labels"] = tokenized["input_ids"].copy()

    return tokenized

# Apply formatting and tokenization
train_dataset = train_dataset.map(
    format_and_tokenize,
    remove_columns=train_dataset.column_names,
    desc="Tokenizing train dataset"
)
val_dataset = val_dataset.map(
    format_and_tokenize,
    remove_columns=val_dataset.column_names,
    desc="Tokenizing val dataset"
)

print(f"Tokenized train dataset: {train_dataset}")
print(f"Tokenized val dataset: {val_dataset}")

Tokenizing train dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

Tokenizing val dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenized train dataset: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 900
})
Tokenized val dataset: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})


## 6. Configure Training (A100 Optimized)

In [66]:
# Training arguments (A100 optimized, same hyperparameters as LoRA for fair comparison)
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,

    # Training hyperparameters (same as LoRA)
    num_train_epochs=config['training']['prompt_tuning_epochs'],
    per_device_train_batch_size=config['training']['prompt_tuning_batch_size'],
    per_device_eval_batch_size=config['training']['prompt_tuning_batch_size'],
    gradient_accumulation_steps=config['training']['prompt_tuning_gradient_accumulation'],

    # Optimizer (slightly higher LR often works better for prompt tuning)
    learning_rate=config['training']['prompt_tuning_learning_rate'],
    weight_decay=0.01,
    optim="paged_adamw_32bit",

    # Learning rate schedule
    lr_scheduler_type="cosine",
    warmup_ratio=config['training']['prompt_tuning_warmup_ratio'],

    # Logging
    logging_dir=f"{OUTPUT_DIR}/logs",
    logging_steps=10,
    logging_first_step=True,

    # Evaluation
    eval_strategy="steps",
    eval_steps=100,

    # Checkpointing
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,

    # Performance - A100 GPU settings
    fp16=False,
    bf16=True,  # ENABLED: A100 supports BF16 natively
    tf32=True,   # ENABLED: TensorFloat-32 for better performance
    gradient_checkpointing=True,

    # Misc
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

print("Training arguments configured (A100 optimized):")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  BF16: {training_args.bf16}, TF32: {training_args.tf32}")

Training arguments configured (A100 optimized):
  Epochs: 3
  Batch size: 12
  Gradient accumulation: 2
  Effective batch size: 24
  Learning rate: 0.0003
  BF16: True, TF32: True


## 7. Initialize Trainer

In [67]:
from transformers import DefaultDataCollator

# Use DefaultDataCollator with padding
# DataCollatorForLanguageModeling doesn't handle variable lengths well
data_collator = DefaultDataCollator(return_tensors="pt")

# Use standard Trainer (not SFTTrainer) for Prompt Tuning
# SFTTrainer tries to call merge_and_unload() which Prompt Tuning doesn't support
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

print("Trainer initialized!")
print("Note: Using standard Trainer (not SFTTrainer) for Prompt Tuning compatibility")

Trainer initialized!
Note: Using standard Trainer (not SFTTrainer) for Prompt Tuning compatibility


## 8. Train Model

In [68]:
# Start training with timing
print("Starting Prompt Tuning training...")
print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 50)

tracker.start_training()
train_result = trainer.train()
tracker.end_training()

print("\n" + "=" * 50)
print(f"Training completed!")
print(f"End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Training time: {tracker.metrics['training_time_seconds']/3600:.2f} hours")

Starting Prompt Tuning training...
Start time: 2025-12-26 15:54:46


Step,Training Loss,Validation Loss
100,3.052100,2.979476



Training completed!
End time: 2025-12-26 16:13:34
Training time: 0.31 hours


In [69]:
# Print training metrics
print("\nTraining metrics:")
print(f"  Final train loss: {train_result.training_loss:.4f}")
print(f"  Total steps: {train_result.global_step}")


Training metrics:
  Final train loss: 5.2226
  Total steps: 114


## 9. Evaluate Model

In [70]:
# Evaluate on validation set
print("Evaluating model...")
eval_results = trainer.evaluate()

print("\nEvaluation metrics:")
for key, value in eval_results.items():
    print(f"  {key}: {value:.4f}")

Evaluating model...



Evaluation metrics:
  eval_loss: 2.9795
  eval_runtime: 11.8096
  eval_samples_per_second: 8.4680
  eval_steps_per_second: 0.7620
  epoch: 3.0000


## 10. Test Generation & Measure Inference Speed

In [71]:
def generate_response(instruction: str, max_new_tokens: int = 256):
    """Generate a response for the given instruction."""
    prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        start_time = time.time()
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
        )
        gen_time = time.time() - start_time

    tokens_generated = outputs.shape[1] - inputs['input_ids'].shape[1]

    generated = tokenizer.decode(outputs[0], skip_special_tokens=False)

    # Extract response
    if "<|start_header_id|>assistant<|end_header_id|>" in generated:
        response = generated.split("<|start_header_id|>assistant<|end_header_id|>")[-1]
        response = response.split("<|eot_id|>")[0].strip()
        return response, tokens_generated, gen_time

    return generated, tokens_generated, gen_time

print("Generation function defined!")

Generation function defined!


In [72]:
# Test generation and measure inference speed
test_instructions = [
    "Explain the concept of machine learning in simple terms.",
    "Write a Python function to calculate the factorial of a number.",
    "What are the main differences between supervised and unsupervised learning?",
]

print("Testing Prompt Tuning model generation:")
print("=" * 50)

total_tokens = 0
total_time = 0

for i, instruction in enumerate(test_instructions):
    print(f"\n[Test {i+1}]")
    print(f"Instruction: {instruction}")
    print(f"\nResponse:")
    response, tokens, gen_time = generate_response(instruction, max_new_tokens=200)
    print(response)
    print(f"\nTokens: {tokens}, Time: {gen_time:.2f}s, Speed: {tokens/gen_time:.1f} tok/s")
    print("-" * 50)

    total_tokens += tokens
    total_time += gen_time

# Log inference speed
tracker.log_inference_speed(total_tokens, total_time)
print(f"\nAverage inference speed: {tracker.metrics['inference_tokens_per_sec']:.1f} tokens/sec")

Testing Prompt Tuning model generation:

[Test 1]
Instruction: Explain the concept of machine learning in simple terms.

Response:


/usr/local/lib/python3.12/dist-packages/peft/peft_model.py:2141: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")


<|end_of_text|>

Tokens: 1, Time: 0.77s, Speed: 1.3 tok/s
--------------------------------------------------

[Test 2]
Instruction: Write a Python function to calculate the factorial of a number.

Response:
!<|end_of_text|>

Tokens: 2, Time: 0.21s, Speed: 9.7 tok/s
--------------------------------------------------

[Test 3]
Instruction: What are the main differences between supervised and unsupervised learning?

Response:
What are the differences between a supervised and unsupervised learning?<|end_of_text|>

Tokens: 14, Time: 1.03s, Speed: 13.6 tok/s
--------------------------------------------------

Average inference speed: 8.4 tokens/sec


## 11. Save Model

In [73]:
# Save the final model
FINAL_MODEL_DIR = f"{config['paths']['models_prompt_tuning']}/final"

print(f"Saving final model to: {FINAL_MODEL_DIR}")

trainer.save_model(FINAL_MODEL_DIR)
tokenizer.save_pretrained(FINAL_MODEL_DIR)

print("Model saved!")

Saving final model to: /content/drive/MyDrive/synthetic-instruction-tuner/models/prompt_tuning/final
Model saved!


In [74]:
# Save training configuration
training_config = {
    "method": "prompt_tuning",
    "base_model": BASE_MODEL_ID,
    "training_data_size": len(train_data),
    "validation_data_size": len(val_data),
    "prompt_tuning_config": {
        "num_virtual_tokens": prompt_tuning_config.num_virtual_tokens,
        "init_method": str(prompt_tuning_config.prompt_tuning_init),
    },
    "training_args": {
        "epochs": training_args.num_train_epochs,
        "batch_size": training_args.per_device_train_batch_size,
        "gradient_accumulation_steps": training_args.gradient_accumulation_steps,
        "learning_rate": training_args.learning_rate,
    },
    "results": {
        "train_loss": train_result.training_loss,
        "eval_loss": eval_results["eval_loss"],
        "total_steps": train_result.global_step,
    },
    "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
}

config_path = f"{FINAL_MODEL_DIR}/training_config.json"
with open(config_path, 'w') as f:
    json.dump(training_config, f, indent=2)

print(f"Training config saved to: {config_path}")

Training config saved to: /content/drive/MyDrive/synthetic-instruction-tuner/models/prompt_tuning/final/training_config.json


## 12. Save Efficiency Metrics

In [75]:
# Save efficiency metrics for comparison
METRICS_DIR = f"{PROJECT_ROOT}/evaluation/metrics"
os.makedirs(METRICS_DIR, exist_ok=True)

# Get summary
summary = tracker.get_summary()

# Add training results
summary["train_loss"] = train_result.training_loss
summary["eval_loss"] = eval_results["eval_loss"]

# Save
tracker.save(f"{METRICS_DIR}/prompt_tuning_metrics.json")

print("\n=== Efficiency Metrics Summary ===")
for key, value in summary.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

Metrics saved to /content/drive/MyDrive/synthetic-instruction-tuner/evaluation/metrics/prompt_tuning_metrics.json

=== Efficiency Metrics Summary ===
  method: prompt_tuning
  trainable_params: 61440
  total_params: 1803525120
  trainable_ratio_percent: 0.0034
  peak_memory_gb: 5.9416
  training_time_hours: 0.3134
  inference_tokens_per_sec: 8.4442
  train_loss: 5.2226
  eval_loss: 2.9795


## 13. Comparison Preview

In [76]:
# Load LoRA metrics if available for comparison
lora_metrics_path = f"{METRICS_DIR}/lora_metrics.json"

if os.path.exists(lora_metrics_path):
    with open(lora_metrics_path, 'r') as f:
        lora_metrics = json.load(f)

    print("\n=== Method Comparison (A100 GPU) ===")
    print(f"{'Metric':<30} {'LoRA':>15} {'Prompt Tuning':>15}")
    print("=" * 60)

    comparisons = [
        ("Trainable Params", "trainable_params"),
        ("Trainable Ratio (%)", "trainable_ratio_percent"),
        ("Peak Memory (GB)", "peak_memory_gb"),
        ("Training Time (hours)", "training_time_hours"),
        ("Inference Speed (tok/s)", "inference_tokens_per_sec"),
        ("Train Loss", "train_loss"),
        ("Eval Loss", "eval_loss"),
    ]

    for label, key in comparisons:
        lora_val = lora_metrics.get(key, "N/A")
        pt_val = summary.get(key, "N/A")

        if isinstance(lora_val, (int, float)) and isinstance(pt_val, (int, float)):
            if key == "trainable_params":
                print(f"{label:<30} {lora_val:>15,} {pt_val:>15,}")
            else:
                print(f"{label:<30} {lora_val:>15.4f} {pt_val:>15.4f}")
        else:
            print(f"{label:<30} {str(lora_val):>15} {str(pt_val):>15}")

    print("\n--- Key Findings ---")
    if isinstance(lora_metrics.get('trainable_params'), (int, float)) and isinstance(summary.get('trainable_params'), (int, float)):
        ratio = lora_metrics['trainable_params'] / summary['trainable_params']
        print(f"Prompt Tuning uses {ratio:.1f}x FEWER parameters than LoRA")
    if isinstance(lora_metrics.get('training_time_hours'), (int, float)) and isinstance(summary.get('training_time_hours'), (int, float)):
        speedup = lora_metrics['training_time_hours'] / summary['training_time_hours']
        print(f"Prompt Tuning is {speedup:.1f}x FASTER than LoRA")
else:
    print("LoRA metrics not found. Run 05_sft_training.ipynb with metrics tracking first.")
    print("Full comparison will be available in 09_comparative_analysis.ipynb")


=== Method Comparison (A100 GPU) ===
Metric                                    LoRA   Prompt Tuning
Trainable Params                    12,156,928          61,440
Trainable Ratio (%)                     0.6696          0.0034
Peak Memory (GB)                        5.3088          5.9416
Training Time (hours)                   0.1369          0.3134
Inference Speed (tok/s)                 7.6954          8.4442
Train Loss                                 N/A 5.222607662803249
Eval Loss                                  N/A 2.979475975036621

--- Key Findings ---
Prompt Tuning uses 197.9x FEWER parameters than LoRA
Prompt Tuning is 0.4x FASTER than LoRA


## 14. Cleanup

In [77]:
# Free GPU memory
del model
del trainer
gc.collect()
torch.cuda.empty_cache()

print("Memory cleared!")

Memory cleared!


## ✓ Prompt Tuning Complete!

### Summary:
- Prompt Tuning model saved to `models/prompt_tuning/final/`
- Efficiency metrics saved for comparison
- Only soft prompt embeddings were trained (minimal parameters)

### Key Differences from LoRA (A100):
- **Parameters**: ~0.01% vs LoRA's ~0.67%
- **Training**: 2-3x faster due to fewer gradients
- **Memory**: Lower peak usage (~15-20GB vs ~20-25GB)
- **Performance**: Often slightly lower quality than LoRA but much more efficient

### Next Steps:
1. Run `09_comparative_analysis.ipynb` for full comparison
2. Compare benchmark scores between methods
3. Include results in your final report

### Training Stats (A100):
- **Training time**: 1-2 hours (2-3x faster than LoRA)
- **Cost**: ~5-10 compute units (half of LoRA)
- **Batch size**: 12 (effective: 24)
- **Precision**: BF16 + TF32